<a href="https://colab.research.google.com/github/InduwaraGayashan001/EN3150-Assignment-03-CNN/blob/main/Assignment_03_Neural_Nexus_Part_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [6]:
# Define image size and batch size
img_size = (224, 224)
batch_size = 32

# Paths to your training and validation directories
train_dir = '/content/drive/My Drive/EN3150-Assignment-03-CNN/Images/train'
validation_dir = '/content/drive/My Drive/EN3150-Assignment-03-CNN/Images/validation'
test_dir = '/content/drive/My Drive/EN3150-Assignment-03-CNN/Images/test'

# Data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse'
)

Found 5467 images belonging to 5 classes.
Found 1823 images belonging to 5 classes.


In [ ]:
# Load pre-trained VGG16 model without the top layers
base_model_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top
x = base_model_vgg.output
x = GlobalAveragePooling2D()(x)
x = Dense(5, activation='softmax')(x)  # 5 classes

# Create the model
model_vgg = Model(inputs=base_model_vgg.input, outputs=x)

# Freeze the base model layers
for layer in base_model_vgg.layers:
    layer.trainable = False

# Compile the model
model_vgg.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history_vgg = model_vgg.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    verbose=1
)


In [ ]:
save_path = '/content/drive/My Drive/models/model_vgg.h5'
model_vgg.save(save_path)
print("VGG Model saved successfully!")

In [ ]:
# Plot Training and Validation Loss
train_loss = history_vgg.history['loss']
val_loss = history_vgg.history['val_loss']
epochs = range(1, len(train_loss) + 1)

plt.figure(figsize=(10, 6))
plt.plot(epochs, train_loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.title('VGG16 Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Load test data generator
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False
)

# Evaluate the model on the test data
test_loss, test_accuracy = model_vgg.evaluate(test_generator, verbose=1)
print(f"Test Accuracy: {test_accuracy:.2f}")